In [1]:
import unittest
import os
import pickle
import numpy as np
import geopandas as gpd
import pandas as pd
import shapely

import sandpyper
from sandpyper.sandpyper import ProfileSet, ProfileDynamics
from sandpyper.common import get_sil_location, get_opt_k, create_transects, sensitivity_tr_rbcd

pd.options.mode.chained_assignment = None  # default='warn'

C:\anaconda3\envs\sandpyper_env\lib\site-packages\pysal\explore\segregation\network\network.py:15: UserWarning: You need pandana and urbanaccess to work with segregation's network module
You can install them with  `pip install urbanaccess pandana` or `conda install -c udst pandana urbanaccess`
  warn(
C:\anaconda3\envs\sandpyper_env\lib\site-packages\pysal\model\spvcm\abstracts.py:10: UserWarning: The `dill` module is required to use the sqlite backend fully.
  from .sqlite import head_to_sql, start_sql


In [2]:
# create global variables used across the test analysis
loc_codes=["mar","leo"]

loc_full={'mar': 'Marengo',
         'leo': 'St. Leonards'}

loc_specs={'mar':{'thresh':6,
       'min_points':6}}

loc_search_dict = {'leo': ['St', 'Leonards', 'leonards', 'leo'], 'mar': ['Marengo', 'marengo', 'mar'] }
crs_dict_string = {'mar': {'init': 'epsg:32754'}, 'leo':{'init': 'epsg:32755'} }

labels=["Undefined", "Small", "Medium", "High", "Extreme"]

appendix=["_deposition", "_erosion"]

relabel_dict={"Undefined_erosion":"ue",
            "Small_erosion":"se",
            "Medium_erosion":"me",
            "High_erosion":"he",
            "Extreme_erosion":"ee",
             "Undefined_deposition":"ud",
             "Small_deposition":"sd",
             "Medium_deposition":"md",
             "High_deposition":"hd",
             "Extreme_deposition":"ed"
            }

transects_spacing=20

feature_set=["band1","band2","band3","distance"]

water_dict={'leo_20180606':[0,9,10],
'leo_20180713':[0,3,4,7],
'leo_20180920':[0,2,6,7],
'leo_20190211':[0,2,5],
'leo_20190328':[2,4,5],
'leo_20190731':[0,2,8,6],
'mar_20180601':[1,6],
'mar_20180621':[4,6],
'mar_20180727':[0,5,9,10],
'mar_20180925':[6],
'mar_20181113':[1],
'mar_20181211':[4],
'mar_20190205':[],
'mar_20190313':[],
'mar_20190516':[4,7]}

no_sand_dict={'leo_20180606':[5],
'leo_20180713':[],
'leo_20180920':[],
'leo_20190211':[1],
'leo_20190328':[],
'leo_20190731':[1],
'mar_20180601':[4,5],
'mar_20180621':[3,5],
'mar_20180727':[4,7],
'mar_20180925':[5],
'mar_20181113':[0],
'mar_20181211':[0],
'mar_20190205':[0,5],
'mar_20190313':[4],
'mar_20190516':[2,5]}

veg_dict={'leo_20180606':[1,3,7,8],
'leo_20180713':[1,5,9],
'leo_20180920':[1,4,5],
'leo_20190211':[4],
'leo_20190328':[0,1,6],
'leo_20190731':[3,7],
'mar_20180601':[0,7],
'mar_20180621':[1,7],
'mar_20180727':[1,3],
'mar_20180925':[1,3],
'mar_20181113':[3],
'mar_20181211':[2],
'mar_20190205':[3],
'mar_20190313':[1,5],
'mar_20190516':[0]}

sand_dict={'leo_20180606':[2,4,6],
'leo_20180713':[2,6,8],
'leo_20180920':[3],
'leo_20190211':[3],
'leo_20190328':[3],
'leo_20190731':[4,5],
'mar_20180601':[2,3],
'mar_20180621':[0,2],
'mar_20180727':[2,6,8],
'mar_20180925':[0,4,2],
'mar_20181113':[2,4],
'mar_20181211':[3,1],
'mar_20190205':[1,2,4],
'mar_20190313':[0,2,3],
'mar_20190516':[1,3,6]}


l_dicts={'no_sand': no_sand_dict,
         'sand': sand_dict,
        'water': water_dict,
        'veg':veg_dict}

In [5]:
from pathlib import Path

In [6]:
test_data_folder = r"C:\my_packages\sandpyper\examples\test_data"

# the paths to the DSM, orthophotos and transect directories
dirNameDSM=Path(test_data_folder + r"\dsm_1m")
dirNameOrtho=Path(test_data_folder + r"\orthos_1m")
dirNameTrans=Path(test_data_folder + r"\transects")

# path to the LoD transects

lod_mode=Path(test_data_folder + r"\lod_transects")

In [3]:
dsms_dir_path = os.path.abspath("examples/test_data/dsm_1m/")
orthos_dir_path = os.path.abspath("examples/test_data/orthos_1m")

transects_path = os.path.abspath("examples/test_data/transects")
lod_mode=os.path.abspath("examples/test_data/lod_transects/")

label_corrections_path=os.path.abspath("examples/test_data/clean/label_corrections.gpkg")
watermasks_path=os.path.abspath("examples/test_data/clean/watermasks.gpkg")
shoremasks_path=os.path.abspath("examples/test_data/clean/shoremasks.gpkg")


In [8]:
P = ProfileSet(dirNameDSM=dirNameDSM,
                        dirNameOrtho=dirNameOrtho,
                        dirNameTrans=dirNameTrans,
                        transects_spacing=transects_spacing,
                        loc_codes=loc_codes,
                        loc_search_dict=loc_search_dict,
                        crs_dict_string=crs_dict_string,
                        check="all")

C:\anaconda3\envs\sandpyper_env\lib\site-packages\geopandas\geodataframe.py:422: RuntimeWarning: Sequential read of iterator was interrupted. Resetting iterator. This can negatively impact the performance.
  for feature in features_lst:


C:\my_packages\sandpyper\examples\test_data\dsm_1m
C:\my_packages\sandpyper\examples\test_data\orthos_1m
dsm from leo = 6

ortho from leo = 6

dsm from mar = 9

ortho from mar = 9


umber OF DATASETS TO PROCESS: 30
